### Create a training pipeline [no image preprocessing]

In [1]:
## import libraries -test

import numpy as np 
import pandas as pd
from pydicom import dcmread
import os
import scipy.ndimage
import matplotlib.pyplot as plt

from torch.utils.data import Dataset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch

from CT_Seg_Network.utils.dataset import *
from CT_Seg_Network.models.Unet import UNet
from CT_Seg_Network.utils.iou import IoU
from CT_Seg_Network.utils.Loss import *
from CT_Seg_Network.models.ResUnet import ResUNet
from tqdm import tqdm

In [2]:
## prepare the dataset by dividing the dataset into train and validation

IMAGE_FOLDER = 'coding_test_files/dicom_series/'
SEG_FOLDER ='coding_test_files/segmentation_data/'
patients = os.listdir(IMAGE_FOLDER)
patients.sort()

## split percentage
percentage = .8
train, val = np.split(patients, [int(len(patients)*percentage)])

def load_list(folder, list_patients):
    folder_list = []
    for i in list_patients:
        for j in os.listdir(folder + i):
            folder_list.append(folder + i + '/' + j)
    return folder_list

train_list_im = load_list(IMAGE_FOLDER, train)
val_list_im = load_list(IMAGE_FOLDER, val)

In [3]:
## create a dataset loader that will handle loading the images from dicom files directly
## and labels from segmentation data

class CSTO_CTDataset(Dataset):
    def __init__(self, image_addr_list, seg_folder=None, transforms=None):

        ## takes the address of the images
        self.images = image_addr_list
        if seg_folder == None:
            self.seg_folder = 'coding_test_files/segmentation_data/'
        else:
            self.seg_folder = seg_folder

        self.transform = transforms

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        image_addr = self.images[idx]

        img = dcmread(image_addr).pixel_array
        img = np.expand_dims(img, axis=0).astype(np.float32)
        img = torch.from_numpy(img).float()

        ## extract the sample name from the image_name
        last_slash = image_addr.find('dicom_series/') + 13
        sample = image_addr[last_slash:-4]
        label_addr = self.seg_folder + sample +'.npz'
        
        if os.path.isfile(label_addr):
            label = torch.from_numpy(np.transpose(np.load(label_addr)['arr_0'], (-1, 0, 1)))#.to(torch.float32)
        else:
            label = torch.zeros([4, img.shape[1], img.shape[2]])#.to(torch.float32)
    
        
        if self.transform:
            img, label = self.transform((img, label))

        sample = {'img': img,
                  'label': label}
        
        return sample
    

In [4]:
## load custom transforms that manipulates both image and label

transform_train = transforms.Compose([
            RandomFlip(),
            Resize(128),
            Normalize(mean=[0.5],
                      std=[0.5])
        ])

transform_val = transforms.Compose([
            Resize(128),
            Normalize(mean=[0.5],
                      std=[0.5])
        ])


## create sets for train and validation
trainset = CSTO_CTDataset(train_list_im, transforms=transform_train)
valset = CSTO_CTDataset(val_list_im, transforms=transform_val)

## create the dataloader for train and validation
trainloader = DataLoader(trainset, batch_size=64, shuffle=True)
valloader = DataLoader(valset, batch_size=16)

In [6]:
## create functions for the training and evaluation proper

def train(model,train_loader,optimizer,LOSS_FUNC,EPOCH,PRINT_INTERVAL, epoch, device):
    losses = []
    for i, batch in enumerate(tqdm(train_loader)):
        img, label = batch['img'].to(device), batch['label'].to(device)
        output = model(img)
        optimizer.zero_grad()

        loss = LOSS_FUNC(output, label)

        loss.backward()
        losses.append(loss.item())
        optimizer.step()
        if (i + 1) % PRINT_INTERVAL == 0:
            tqdm.write('Epoch [%d/%d], Iter [%d/%d], Loss: %.4f'
                       % (epoch + 1, EPOCH, i + 1, len(train_loader), loss.item()))
    return np.mean(losses)

def eval(model,val_loader,LOSS_FUNC, device):
    losses = []
    for i, batch in enumerate(val_loader):
        img, label = batch['img'].to(device), batch['label'].to(device)
        output = model(img)
        loss = LOSS_FUNC(output, label)
        losses.append(loss.item())
    return np.mean(losses)

In [7]:
batch_size = 16
print("Train set {}\nValidation set {}".format(len(trainset),len(valset)))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## initialize model type
model = ResUNet(out_classes=4).to(device)


if torch.cuda.device_count() > 1:
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    # dim = 0 [30, xxx] -> [10, ...], [10, ...], [10, ...] on 3 GPUs
    model = nn.DataParallel(model).to(device)

## init optimizer, scheduler, and loss function
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, momentum=0.9)
lr_sheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)
LOSS_FUNC = DiceCELoss().to(device)
PRINT_INTERVAL = 5
EPOCH= 50



## start training
val_loss_epoch = []
for epoch in range(EPOCH):

    model.train()
    train_loss = train(model, trainloader, optimizer, LOSS_FUNC, EPOCH, PRINT_INTERVAL, epoch, device)
    val_loss = eval(model, valloader, LOSS_FUNC, device)
    val_loss_epoch.append(val_loss)
    lr_sheduler.step()
    tqdm.write('Epoch [%d/%d], Average Train Loss: %.4f, Average Validation Loss: %.4f'
                % (epoch + 1, EPOCH, train_loss, val_loss))


    ## save the model with the best val result
    if val_loss == np.min(val_loss_epoch):
        print('Model saved')
        state = {
            'epoch': epoch + 1,
            'state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict(),
        }
        checkpoint_addr = 'CT_Seg_Network/weights/'
        torch.save(state, os.path.join(checkpoint_addr,'best.pth'))

Train set 3300
Validation set 1103


 10%|▉         | 5/52 [00:13<01:52,  2.39s/it]

Epoch [1/50], Iter [5/52], Loss: 1.0552


 19%|█▉        | 10/52 [00:24<01:33,  2.24s/it]

Epoch [1/50], Iter [10/52], Loss: 1.0395


...

Epoch [20/50], Iter [45/52], Loss: 0.8942


 96%|█████████▌| 50/52 [01:57<00:04,  2.47s/it]

Epoch [20/50], Iter [50/52], Loss: 0.9001


100%|██████████| 52/52 [02:01<00:00,  2.34s/it]


Epoch [20/50], Average Train Loss: 0.9028, Average Validation Loss: 1.0010
